In [1]:
# calculate dM/M0 time series for every ROI
import os
import sys
import pandas as pd
import subprocess
import glob
import re
import numpy as np

local_dir = '../data_refine'
roi_dir = '../rois'


In [2]:
def refine_list(lst):
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra \n\n
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [3]:
cols = ['subject_id', 'model', 'scalar','GM','WM','acc','caudate', 'thalamus', 'lv', 'csf']

df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    # skip the uneven sessions for now
    if '4907' in root or '5509' in root or '0000' in root or '7586' in root:
        continue
    if 'sub-' in root and 'hex' not in root and 'baseline' not in root:
        scalar_maps = glob.glob(root+"/**/*acq-glymph*.nii.gz", recursive=True)
        sub_name = root.split('/')[4].split('sub-')[1]
        model = root.split('/')[2]    
        scalar = root.split('/')[3]
            
        if len(scalar_maps) > 0:
            # merge scalar volumes into one time series volume
            scalar_ts = os.path.join(root, 'sub-{}_{}_{}_ts.nii.gz'.format(sub_name, model, scalar))
            scalar_maps.sort()
            print(scalar_ts)
            pattern = "x|(.*?)|_space"
            for s in scalar_maps:
                substring = s.split('full')[1].split('_space')[0]
                print(substring)
            print('')
            
            fslmerge_inputs = ' '.join(scalar_maps)
            cmd1="fslmerge -t {} {}".format(scalar_ts, fslmerge_inputs)
            subprocess.check_output(cmd1, shell=True)
            
            for mask in glob.glob(roi_dir+"/*mask.nii.gz"):
                cmd2="fslmeants --transpose -i '{}' -m '{}'".format(scalar_ts, mask)
                result = refine_list(subprocess.check_output(cmd2, shell=True))
                if 'gm' in mask:
                    gm = result
                elif 'wm' in mask:
                    wm = result
                if 'acc' in mask:
                    acc = result
                elif 'caudate' in mask:
                    caudate = result
                elif 'thal' in mask:
                    thalamus = result
                elif 's1' in mask:
                    s1 = result
                elif 'csf' in mask:
                    csf = result
                elif 'lv' in mask:
                    lv = result

            df.loc[len(df.index)] = [sub_name, model, scalar, gm, wm, acc, caudate, thalamus, lv, csf]

df.to_csv('ts_data.csv', index=False)
df.head()

../data_refine/GQI/iso/sub-MB5330/sub-MB5330_GQI_iso_ts.nii.gz
1501
1504
1514
1517
1527
1530
1553
1556
1606
1609
1619
1621

../data_refine/GQI/iso/sub-AG6470/sub-AG6470_GQI_iso_ts.nii.gz
1505
1508
1519
1522
1532
1535
1557
1600
1610
1613
1622
1625

../data_refine/GQI/iso/sub-AM5508/sub-AM5508_GQI_iso_ts.nii.gz
1052
1055
1108
1111
1123
1126
1150
1153
1206
1208
1220
1223

../data_refine/GQI/iso/sub-AA5325/sub-AA5325_GQI_iso_ts.nii.gz
1008
1010
1021
1024
1034
1037
1104
1107
1117
1120
1129
1132

../data_refine/GQI/iso/sub-KS5471/sub-KS5471_GQI_iso_ts.nii.gz
1114
1117
1126
1129
1141
1144
1205
1208
1218
1221
1231
1234

../data_refine/GQI/iso/sub-PER5474/sub-PER5474_GQI_iso_ts.nii.gz
1412
1416
1425
1428
1438
1441
1510
1513
1523
1526
1537
1540

../data_refine/GQI/iso/sub-CM5268/sub-CM5268_GQI_iso_ts.nii.gz
1101
1104
1112
1114
1122
1125
1150
1153
1202
1205
1215
1218

../data_refine/GQI/iso/sub-CKA6605/sub-CKA6605_GQI_iso_ts.nii.gz
1127
1130
1141
1144
1155
1158
1221
1224
1235
1238
1248
1251

../d

,subject_id,model,scalar,GM,WM,acc,caudate,thalamus,lv,csf
0,MB5330,GQI,iso,"[0.200636, 0.207349, 0.192243, 0.205165, 0.216...","[0.207202, 0.215115, 0.19871, 0.212884, 0.2242...","[0.208031, 0.219377, 0.20253, 0.218277, 0.2291...","[0.209803, 0.221403, 0.204345, 0.220674, 0.235...","[0.185593, 0.197859, 0.18052, 0.195454, 0.2081...","[0.070318, 0.07407, 0.068283, 0.074124, 0.0773...","[0.131092, 0.135645, 0.124846, 0.133439, 0.141..."
1,AG6470,GQI,iso,"[0.244378, 0.236701, 0.251958, 0.24621, 0.2342...","[0.252908, 0.246563, 0.261418, 0.255285, 0.243...","[0.274828, 0.270195, 0.286724, 0.279462, 0.266...","[0.302385, 0.297936, 0.307736, 0.303713, 0.286...","[0.259048, 0.256311, 0.265161, 0.26215, 0.2482...","[0.099785, 0.097999, 0.101118, 0.100783, 0.095...","[0.163456, 0.158294, 0.168966, 0.164618, 0.156..."
2,AM5508,GQI,iso,"[0.207368, 0.197094, 0.186958, 0.189082, 0.186...","[0.201513, 0.192101, 0.183694, 0.183691, 0.181...","[0.232884, 0.219202, 0.212976, 0.213258, 0.211...","[0.264339, 0.253393, 0.245846, 0.241851, 0.243...","[0.200452, 0.190934, 0.181494, 0.178636, 0.178...","[0.115654, 0.110547, 0.107433, 0.102453, 0.103...","[0.149621, 0.143511, 0.137429, 0.136356, 0.135..."
3,AA5325,GQI,iso,"[0.21421, 0.197221, 0.220012, 0.203542, 0.2038...","[0.22085, 0.20253, 0.226644, 0.209422, 0.20945...","[0.24506, 0.22495, 0.250746, 0.230565, 0.22943...","[0.270753, 0.251591, 0.283037, 0.260538, 0.259...","[0.218984, 0.199904, 0.224873, 0.204398, 0.205...","[0.10462, 0.09059, 0.106455, 0.094764, 0.09566...","[0.153667, 0.138236, 0.157579, 0.142249, 0.144..."
4,KS5471,GQI,iso,"[0.18363, 0.186319, 0.196978, 0.184978, 0.1902...","[0.186871, 0.191168, 0.20084, 0.18917, 0.19432...","[0.212126, 0.215157, 0.22886, 0.217496, 0.2204...","[0.213795, 0.217234, 0.225773, 0.212375, 0.213...","[0.17819, 0.186217, 0.189658, 0.180295, 0.1813...","[0.062188, 0.065347, 0.067338, 0.063574, 0.064...","[0.121793, 0.122338, 0.130036, 0.122711, 0.125..."


### Normalize time series data to baseline

In [8]:
def normalize_ts(ts, fullbaseline):
    # normalize ts based on the first baseline timepoint
    normalized_ts = []
    
    if fullbaseline:
        baseline = sum(ts[0:6])/6
    else:
        baseline = ts[0]
    
    for tpoint in ts:
        norm_tpoint = 100*(tpoint - baseline)/baseline
        normalized_ts.append(norm_tpoint)
        
    return normalized_ts

In [9]:
# TESTING THE FUNCTION
ts = [0.200636, 0.207349, 0.192243, 0.205165, 0.216784, 0.202897, 0.219422, 0.219491, 0.222456, 0.221574, 0.215319, 0.227749]
new_ts = normalize_ts(ts, True)
print(new_ts)

[-1.7352421159864717, 1.552559273970378, -5.845850944514386, 0.4829096038279936, 6.1735046209453355, -0.6278804382429305, 7.46550820603489, 7.499302082976194, 8.951459258787612, 8.519485353537808, 5.455996943858076, 11.543792456618936]


In [12]:
norm_df = df.copy()
for row, row in df.iterrows(): #iterate over rows
    row_list = [row.subject_id, row.model, row.scalar] 
    sub_id = row.subject_id
    for c, value in row.items():
        if type(value) == list:
            norm_ts = normalize_ts(value,False) 
            row_list.append(norm_ts)
            
    norm_df.loc[len(norm_df.index)] = row_list

norm_df = norm_df.tail(40) # only get the newly created rows
norm_df.reset_index(drop=True)
norm_df.to_csv('ts_data_norm_t0.csv', index=False)


In [13]:
norm_df

,subject_id,model,scalar,GM,WM,acc,caudate,thalamus,lv,csf
40,MB5330,GQI,iso,"[0.0, 3.3458601646763273, -4.183197432165719, ...","[0.0, 3.818978581287827, -4.098416038455227, 2...","[0.0, 5.453994837307898, -2.644317433459439, 4...","[0.0, 5.528996248862027, -2.6014880626111117, ...","[0.0, 6.609085471973619, -2.7334005054069896, ...","[0.0, 5.335760402741817, -2.8939958474359466, ...","[0.0, 3.473133371983036, -4.7645928050529305, ..."
41,AG6470,GQI,iso,"[0.0, -3.141444810907699, 3.1017522035535126, ...","[0.0, -2.5088174355892328, 3.3648599490723745,...","[0.0, -1.6857816525244873, 4.328525477753344, ...","[0.0, -1.47130314003672, 1.7695983597069942, 0...","[0.0, -1.0565609462338985, 2.359794323831869, ...","[0.0, -1.7898481735731784, 1.335872125068899, ...","[0.0, -3.1580364134690684, 3.3709377447141833,..."
42,AM5508,GQI,iso,"[0.0, -4.954477064928053, -9.84240577138227, -...","[0.0, -4.670666408618801, -8.842605687970504, ...","[0.0, -5.875027910891259, -8.54846189519246, -...","[0.0, -4.140894835797976, -6.995940818418767, ...","[0.0, -4.748268912258296, -9.457625765769363, ...","[0.0, -4.415757345184775, -7.108271222785209, ...","[0.0, -4.083651359100664, -8.148588767619524, ..."
43,AA5325,GQI,iso,"[0.0, -7.93100228747491, 2.7085570234816307, -...","[0.0, -8.295223002037584, 2.6235001131990137, ...","[0.0, -8.206153595037945, 2.3202481025055186, ...","[0.0, -7.07729923583488, 4.5369765062621505, -...","[0.0, -8.71296533080043, 2.6892375698681077, -...","[0.0, -13.410437774804052, 1.7539667367616032,...","[0.0, -10.041843727020115, 2.5457645428101014,..."
44,KS5471,GQI,iso,"[0.0, 1.4643576757610548, 7.268964766105756, 0...","[0.0, 2.2994472122480185, 7.475210171722729, 1...","[0.0, 1.4288677484136683, 7.888707654884361, 2...","[0.0, 1.608550246731681, 5.602563203068354, -0...","[0.0, 4.5047421291879495, 6.435826926314612, 1...","[0.0, 5.079758152698273, 8.28134045153405, 2.2...","[0.0, 0.4474805612802081, 6.768040856206854, 0..."
45,PER5474,GQI,iso,"[0.0, -4.431184340746944, -4.033243199595859, ...","[0.0, -3.8688651543996713, -3.6659836726791704...","[0.0, -3.416690898517003, -1.627634747727791, ...","[0.0, -3.1999249773368943, -1.0058594089186506...","[0.0, -3.3768484143294195, -1.1514266221889258...","[0.0, -4.244889956374693, -0.7891770011273913,...","[0.0, -5.674921237954042, -4.455267790956267, ..."
46,CM5268,GQI,iso,"[0.0, -3.860182509575129, 0.6696887111573682, ...","[0.0, -4.100662540556754, -0.2644709218311199,...","[0.0, -3.124551923604776, -0.2560436544189097,...","[0.0, -4.3251098712713905, 0.17298881027335347...","[0.0, -4.0811265523646565, 0.6321296935629725,...","[0.0, -1.4876524843796473, -1.8361882092914454...","[0.0, -3.9790384099547773, 1.2855656816115186,..."
47,CKA6605,GQI,iso,"[0.0, 0.7714299295798546, -4.691054536564827, ...","[0.0, -0.20317767266836026, -5.715107011262972...","[0.0, -0.49256253640867326, -5.698498664774857...","[0.0, 0.35858185586892694, -5.660272788237185,...","[0.0, -0.1260571459061591, -6.395565989915437,...","[0.0, -0.8355036355698736, -6.170683607569209,...","[0.0, 0.3607814197568877, -4.458520386200117, ..."
48,EP5270,GQI,iso,"[0.0, -3.4688370902143815, -3.7854392547293374...","[0.0, -2.8624073782121937, -3.5427833832571354...","[0.0, -1.760688173548548, -2.1151010514248734,...","[0.0, -1.9100428934039309, -2.309792521878705,...","[0.0, -1.5197505089263892, -3.3065897506651165...","[0.0, -1.7600461980742437, -4.221851186964114,...","[0.0, -2.702319212787246, -3.494072492249179, ..."
49,MB5330,GQI,md,"[0.0, 0.01438950593888327, 0.1596978203752827,...","[0.0, -0.06401158190864947, 0.0535423979516248...","[0.0, -0.2307878586262279, -0.8322794795715381...","[0.0, -1.320043492275799, -2.282863369785377, ...","[0.0, -1.9398513013036995, -1.1542780660550709...","[0.0, -0.3133026030422454, -0.4765799381504395...","[0.0, 0.05867473470048989, 0.4192820233605892,..."
